In [1]:
# advanced_dataframes_exercises

In [1]:
import numpy as np
import pandas as pd

In [2]:
from env import host, user, password

In [4]:
url = f'mysql+pymysql://{user}:{password}@{host}/employees'


In [5]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)


,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


In [6]:
sql = '''
SELECT
    emp_no,
    first_name,
    last_name
FROM employees
WHERE gender = 'F'
LIMIT 100
'''

In [7]:
pd.read_sql(sql,url)

,emp_no,first_name,last_name
0,10002,Bezalel,Simmel
1,10006,Anneke,Preusig
2,10007,Tzvetan,Zielinski
3,10009,Sumant,Peac
4,10010,Duangkaew,Piveteau
...,...,...,...
95,10245,Ramalingam,Gente
96,10247,Heon,Riefers
97,10248,Frederique,Tempesti
98,10252,Shirish,Wegerle


In [8]:
query = '''
SELECT
    t.title as title,
    d.dept_name as dept_name
FROM titles t
JOIN dept_emp USING (emp_no)
JOIN departments d USING (dept_no)
LIMIT 100
'''

title_dept = pd.read_sql(query, url)
title_dept.head()


,title,dept_name
0,Staff,Customer Service
1,Senior Staff,Customer Service
2,Staff,Customer Service
3,Senior Staff,Customer Service
4,Staff,Customer Service


In [3]:
def get_db_url(db_name, user=user, host=host, password=password):
    '''
    get_db_url accepts a database name, username, hostname, password 
    and returns a url connection string formatted to work with codeup's 
    sql database.
    Default values from env.py are provided for user, host, and password.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

In [10]:
connection_string = get_db_url(user, host, password, 'employees')

In [11]:
pd.read_sql(query, connection_string)


,title,dept_name
0,Staff,Customer Service
1,Senior Staff,Customer Service
2,Staff,Customer Service
3,Senior Staff,Customer Service
4,Staff,Customer Service
...,...,...
95,Senior Staff,Customer Service
96,Staff,Customer Service
97,Senior Staff,Customer Service
98,Senior Staff,Customer Service


In [ ]:
# If I intentionally make an error in the connection string (in this case, 
# I misspelled the database name), I get this error when I run the sql query:
# OperationalError: (pymysql.err.OperationalError) (1044, "Access denied for 
# user 'pagel_2179'@'%' to database 'employes'")

connection_string = get_db_url(user, host, password, 'employes')
pd.read_sql(query, connection_string)

In [ ]:
# If I intentionally make an error in the SQL query (in this case,
# I removed the comma after title in the SELECT statement), I get this error:
# ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error 
# in your SQL syntax; ...
# The error looks very similar to the error that mysql would give if I 
# ran it in MySQLWorkBench

connection_string = get_db_url(user, host, password, 'employees')
error_query = '''
SELECT
    t.title as title
    d.dept_name as dept_name
FROM titles t
JOIN dept_emp USING (emp_no)
JOIN departments d USING (dept_no)
LIMIT 100
'''
pd.read_sql(error_query, connection_string)

In [12]:
# Read the employees and titles tables into two separate DataFrames.

employees_db_connect_string = get_db_url(user, host, password, 'employees')
employees_query = '''
    SELECT *
    FROM employees
'''
titles_query = '''
    SELECT *
    FROM titles
'''


In [13]:
employees_df = pd.read_sql(employees_query, employees_db_connect_string)

In [14]:
titles_df = pd.read_sql(titles_query, employees_db_connect_string)

In [ ]:
employees_df

In [ ]:
titles_df

In [ ]:
# How many rows and columns do you have in each DataFrame? Is that what you expected?

# employees_df had 300024 rows x 6 columns
# titles_df had 443308 rows x 4 columns
employees_df.shape

In [15]:
titles_df.shape

(443308, 4)

In [ ]:
# Display the summary statistics for each DataFrame.

employees_df.info()
#employees_df.describe()
titles_df.info()


In [ ]:
# How many unique titles are in the titles DataFrame?

print("The number of unique titles in the titles DataFrame = ")
titles_df['title'].nunique()

In [ ]:
# What is the oldest date in the to_date column?

print("The oldest date in the to_date columns are: ")
print(titles_df['to_date'].min())

In [16]:
# What is the most recent date in the to_date column?

print("The most recent date in the to_date columns are: ")
print(titles_df['to_date'].max())



The most recent date in the to_date columns are: 
9999-01-01


In [19]:
# The most recent date in the to_date column that is not 9999-01-01 ?

(titles_df['to_date'][titles_df['to_date'] != titles_df['to_date'].max()]).max()

datetime.date(2002, 8, 1)

In [ ]:
###### Exercise Part 2 ########

In [4]:
#     Copy the users and roles DataFrames from the examples above.

# Create the users DataFrame

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [5]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [6]:
#     What is the result of using a right join on the DataFrames?

pd.merge(users, roles, how='right', left_on='role_id', right_on='id')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


In [7]:
#     What is the result of using an outer join on the DataFrames?

pd.merge(users, roles, how='outer', left_on='role_id', right_on='id', indicator=True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


In [9]:
#     What happens if you drop the foreign keys from the DataFrames and try to merge them?

# It doesn't work very well
users_without_role_id = users.drop(columns=['role_id'])
pd.merge(users_without_role_id, roles, how='outer', on = 'id')

,id,name_x,name_y
0,1,bob,admin
1,2,joe,author
2,3,sally,reviewer
3,4,adam,commenter
4,5,jane,NaN
5,6,mike,NaN


In [10]:
#     Load the mpg dataset from PyDataset.

from pydataset import data 

mpg = data('mpg')
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


In [44]:
#     Output and read the documentation for the mpg dataset.

data('mpg', show_doc=True)
print(mpg.head())

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 


  manufacturer model  displ  year  cyl       trans drv  cty  hwy fl    class
1         audi    a4    1.8  1999    4    auto(l5)   f   18   29  p  compact
2     

In [45]:
#     How many rows and columns are in the dataset?

# this is also in the show_doc=True documentation
mpg.shape

(234, 11)

In [11]:
#     Check out your column names and perform any cleanup you may want on them.

mpg = mpg.rename(columns={'cty': 'city', 'hwy': 'highway', 'class': 'car_class'})
mpg


,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,car_class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


In [14]:
#     Display the summary statistics for the dataset.

mpg.describe(include='all')

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,car_class
count,234,234,234.000000,234.000000,234.000000,234,234,234.000000,234.000000,234,234
unique,15,38,NaN,NaN,NaN,10,3,NaN,NaN,5,7
top,dodge,caravan 2wd,NaN,NaN,NaN,auto(l4),f,NaN,NaN,r,suv
freq,37,11,NaN,NaN,NaN,83,106,NaN,NaN,168,62
mean,NaN,NaN,3.471795,2003.500000,5.888889,NaN,NaN,16.858974,23.440171,NaN,NaN
std,NaN,NaN,1.291959,4.509646,1.611534,NaN,NaN,4.255946,5.954643,NaN,NaN
min,NaN,NaN,1.600000,1999.000000,4.000000,NaN,NaN,9.000000,12.000000,NaN,NaN
25%,NaN,NaN,2.400000,1999.000000,4.000000,NaN,NaN,14.000000,18.000000,NaN,NaN
50%,NaN,NaN,3.300000,2003.500000,6.000000,NaN,NaN,17.000000,24.000000,NaN,NaN
75%,NaN,NaN,4.600000,2008.000000,8.000000,NaN,NaN,19.000000,27.000000,NaN,NaN


In [12]:
#     How many different manufacturers are there?

# Two ways. First, use nunique to count the number of unique manufacturers
mpg.manufacturer.nunique()

# OR use groupby and the 'manufacturer' column and then take the lenth
len(mpg.groupby('manufacturer').manufacturer.count())

15

In [13]:
#     How many different models are there?

# Same two ways
mpg.model.nunique()

len(mpg.groupby('model').model.count())

38

In [82]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,car_class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [15]:
#     Create a column named mileage_difference like you did in the DataFrames exercises; this column 
# should contain the difference between highway and city mileage for each car.

# could do this the same way we did before:
#  mpg['mileage_difference'] = mpg.highway - mpg.city

# However, here is a way using pd.concat:
mileage_diff_df = pd.DataFrame(mpg.highway - mpg.city, columns=['mileage_difference'])
mpg = pd.concat([mpg, mileage_diff_df], axis = 1)
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,car_class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8


In [25]:
#     Create a column named average_mileage like you did in the DataFrames exercises; this is the mean 
# of the city and highway mileage.

average_mileage = pd.DataFrame(((mpg.highway + mpg.city) / 2), columns = ['average_mileage'])
mpg = pd.concat([mpg, average_mileage], axis = 1)
mpg

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,car_class,mileage_difference,is_automatic,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,True,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,False,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,False,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,True,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,True,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,True,23.5
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,False,25.0
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,True,21.0
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,False,22.0


In [23]:
#     Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether 
# the car has an automatic transmission.

# could do this:
# mpg['is_automatic'] = mpg.trans.str.contains('auto')
# mpg
# mpg = mpg.drop(columns = ['is_automatic'])
 
# OR

# is_auto_df = pd.DataFrame(np.where(mpg.trans.str.contains('auto'), True, False), columns = ['is_automatic'])
# is_auto_df
# mpg = pd.concat([mpg, is_auto_df], axis=1)

# is_auto_df has an index that starts with 0 which messes this up

# This works:
mpg.trans.value_counts()
mpg.trans.str.contains('auto')
mpg['is_auto'] = mpg.trans.str.contains('auto')
mpg = mpg.rename(columns = {'is_auto': 'is_automatic'})
mpg


,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,car_class,mileage_difference,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,False


In [26]:
#     Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

mpg

# get all of the average mpg's gouped by manufacturer and sort the highest to lowest
mpg.groupby('manufacturer').average_mileage.mean().sort_values(ascending=False)

# OR to just get the max average mileage with the manufacturer:
# pd.DataFrame(mpg.groupby('manufacturer').average_mileage.mean().sort_values(ascending=False)).iloc[0:1,:]

manufacturer
honda         28.500000
volkswagen    25.074074
hyundai       22.750000
subaru        22.428571
audi          22.027778
toyota        21.720588
pontiac       21.700000
nissan        21.346154
chevrolet     18.447368
ford          16.680000
mercury       15.625000
jeep          15.562500
dodge         15.540541
lincoln       14.166667
land rover    14.000000
Name: average_mileage, dtype: float64

In [27]:
#     Do automatic or manual cars have better miles per gallon?

# manual cars have better mpg's on average
pd.DataFrame(mpg.groupby('is_automatic').average_mileage.mean())

,average_mileage
is_automatic,
False,22.227273
True,19.130573


In [164]:
mpg.groupby('manufacturer').model.count()

manufacturer
audi          18
chevrolet     19
dodge         37
ford          25
honda          9
hyundai       14
jeep           8
land rover     4
lincoln        3
mercury        4
nissan        13
pontiac        5
subaru        14
toyota        34
volkswagen    27
Name: model, dtype: int64

In [168]:
is_auto_mask = np.where(mpg.trans.str.contains('auto'), True, False)
mpg[is_auto_mask]

,manufacturer,model,displ,year,cyl,trans,drv,city,highway,fl,car_class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999.0,4.0,auto(l5),f,18.0,29.0,p,compact,11.0,23.5,False
4,audi,a4,2.0,2008.0,4.0,auto(av),f,21.0,30.0,p,compact,9.0,25.5,True
5,audi,a4,2.8,1999.0,6.0,auto(l5),f,16.0,26.0,p,compact,10.0,21.0,False
7,audi,a4,3.1,2008.0,6.0,auto(av),f,18.0,27.0,p,compact,9.0,22.5,False
9,audi,a4 quattro,1.8,1999.0,4.0,auto(l5),4,16.0,25.0,p,compact,9.0,20.5,False
11,audi,a4 quattro,2.0,2008.0,4.0,auto(s6),4,19.0,27.0,p,compact,8.0,23.0,True
12,audi,a4 quattro,2.8,1999.0,6.0,auto(l5),4,15.0,25.0,p,compact,10.0,20.0,False
14,audi,a4 quattro,3.1,2008.0,6.0,auto(s6),4,17.0,25.0,p,compact,8.0,21.0,False
16,audi,a6 quattro,2.8,1999.0,6.0,auto(l5),4,15.0,24.0,p,midsize,9.0,19.5,True
17,audi,a6 quattro,3.1,2008.0,6.0,auto(s6),4,17.0,25.0,p,midsize,8.0,21.0,True
